In [1]:
pip install selenium beautifulsoup4 requests yt-dlp pytz

Note: you may need to restart the kernel to use updated packages.


In [28]:
# 필요한 라이브러리 임포트
import time
import json
import re
from datetime import datetime, timedelta

# Selenium 관련 임포트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException

# BeautifulSoup 관련 임포트
from bs4 import BeautifulSoup

# yt-dlp 관련 임포트
import yt_dlp
import pytz

# WebDriver 경로 설정 (본인의 ChromeDriver 경로로 변경하세요)
# 예시: CHROME_DRIVER_PATH = '/path/to/chromedriver'
# 만약 환경 변수에 경로가 설정되어 있거나, WebDriver_manager를 사용한다면 필요 없을 수 있습니다.
# CHROME_DRIVER_PATH = 'C:/Users/사용자이름/Downloads/chromedriver-win64/chromedriver.exe' # 예시 (Windows)
# CHROME_DRIVER_PATH = '/usr/local/bin/chromedriver' # 예시 (macOS/Linux)
# 최신 Selenium은 자동으로 드라이버를 관리하는 경향이 있으므로, 아래 주석 처리된 부분을 사용할 수도 있습니다.
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


# 전역 변수 설정
YOUTUBE_MAIN_URL = "https://www.youtube.com/channel/UCcQTRi69dsVYHN3exePtZ1A/videos/videos"
SCROLL_PAUSE_TIME = 2 # 스크롤 한 번 후 대기 시간 (초)
MAX_SCROLLS = 500 # 최대 스크롤 횟수 (무한 스크롤 방지)

# 필터링 조건 (초 단위)
MIN_VIDEO_LENGTH_SECONDS = 60   # 최소 1분
MAX_VIDEO_LENGTH_SECONDS = 300  # 최대 5분 (300초)

# KST 시간대 설정
KST = pytz.timezone('Asia/Seoul')

# 1단계 크롤링 결과를 저장할 리스트
filtered_video_urls_1st_stage = []
# 최종 크롤링 결과를 저장할 리스트
final_video_data = []

# WebDriver 초기화 함수
def initialize_webdriver():
    chrome_options = Options()
    # 크롬 브라우저를 백그라운드에서 실행 (Headless 모드)
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--incognito") # 시크릿 모드

    try:
        # WebDriverManager를 사용하여 ChromeDriver 자동 다운로드 및 설정
        service = ChromeService(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("WebDriver 초기화 완료.")
        return driver
    except Exception as e:
        print(f"WebDriver 초기화 실패: {e}")
        return None

# WebDriver 초기화
driver = initialize_webdriver()

# 시간 파싱 도우미 함수 (1단계에서 사용)
def parse_duration_to_seconds(duration_str):
    """'1:23' 또는 '1:23:45' 형식의 문자열을 초 단위로 변환"""
    parts = list(map(int, duration_str.split(':')))
    if len(parts) == 3: # 시:분:초
        return parts[0] * 3600 + parts[1] * 60 + parts[2]
    elif len(parts) == 2: # 분:초
        return parts[0] * 60 + parts[1]
    return 0

# 1단계 필터링을 위한 시간 확인 함수
def is_within_24_hours(upload_time_str):
    """
    'N분 전', 'N시간 전', '1일 전' 등의 문자열을 파싱하여 24시간 이내인지 대략적으로 판단.
    (정확한 판단은 2단계 yt-dlp에서 수행)
    """
    upload_time_str = upload_time_str.replace(" ", "") # 공백 제거

    if "분전" in upload_time_str:
        minutes = int(re.search(r'(\d+)', upload_time_str).group(1))
        return minutes <= 60 * 24 # 24시간 = 1440분
    elif "시간전" in upload_time_str:
        hours = int(re.search(r'(\d+)', upload_time_str).group(1))
        return hours <= 24
    elif "일전" in upload_time_str:
        days = int(re.search(r'(\d+)', upload_time_str).group(1))
        # 1일 전이라도 정확히 24시간을 넘지 않았을 수 있으므로 포함 (2단계에서 정밀 필터링)
        return days < 1 or (days == 1 and "시간전" not in upload_time_str) # '1일 전' 이면 일단 포함
    return False # 기타 형식 (예: '주 전', '개월 전')은 24시간 초과로 간주

WebDriver 초기화 완료.


In [29]:
# Cell 4: 1단계 - 유튜브 메인 페이지 동영상 목록 크롤링 및 빠른 필터링

def crawl_main_page_and_filter_videos(driver):
    """
    유튜브 메인 페이지에서 동영상 목록을 크롤링하고 1차 필터링합니다.
    yt-dlp 호출 없이 Selenium과 BeautifulSoup으로 빠르게 정보 수집.
    """
    print("1단계 크롤링 시작: 메인 페이지 동영상 목록 필터링")
    try:
        driver.get(YOUTUBE_MAIN_URL)
        print(f"{YOUTUBE_MAIN_URL} 접속 완료.")

        # 페이지 로딩 대기 (동영상 컨텐츠가 나타날 때까지)
        WebDriverWait(driver, 30).until( # 타임아웃을 30초로 늘려 안정성 확보
            EC.presence_of_element_located((By.ID, "contents"))
        )
        print("페이지 컨텐츠 로딩 완료.")
    except TimeoutException:
        print("페이지 컨텐츠 로딩 시간 초과. 네트워크 상태를 확인하거나 타임아웃을 늘리세요.")
        return
    except WebDriverException as e:
        print(f"유튜브 메인 페이지 접속 중 오류 발생: {e}")
        return

    seen_video_ids = set() # 중복 방지를 위해 이미 처리한 영상 ID 저장
    scroll_count = 0
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    stop_scrolling = False
    no_change_count = 0 # 높이 변화가 없는 횟수를 세는 카운터

    while not stop_scrolling and scroll_count < MAX_SCROLLS:
        print(f"\n스크롤 {scroll_count + 1}회 시작...")
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME + 1) # 스크롤 후 새로운 컨텐츠 로딩 대기 (기존 2초 + 1초 = 3초)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_height == last_height:
            no_change_count += 1
            print(f"높이 변화 없음 (연속 {no_change_count}회).")
            if no_change_count >= 3: # 3번 연속 변화가 없으면 중단
                print("3회 연속 스크롤 높이 변화 없음. 더 이상 로드할 컨텐츠가 없거나 로딩이 매우 느립니다. 중단합니다.")
                break
        else:
            no_change_count = 0 # 높이 변화가 있으면 카운터 초기화
        
        last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 동영상 요소를 찾는 더 견고한 방법
        video_containers = soup.select('ytd-rich-item-renderer, ytd-video-renderer') 
        
        if not video_containers:
            print("경고: 동영상 컨테이너 요소를 찾을 수 없습니다. 셀렉터를 확인하세요.")
            if no_change_count >= 3:
                break
            continue

        current_scroll_videos_count = 0
        for video in video_containers:
            try:
                # 영상 링크 및 ID 추출
                link_tag = video.select_one('a#thumbnail')
                if not link_tag or 'href' not in link_tag.attrs:
                    continue
                video_url_suffix = link_tag['href']
                if not video_url_suffix.startswith('/watch?v='): # 유효한 동영상 링크인지 확인
                    continue
                video_id = video_url_suffix.split('v=')[1].split('&')[0] # 고유 ID 추출

                if video_id in seen_video_ids:
                    continue # 이미 처리한 영상은 건너뛰기
                
                # 업로드 시간 추출 (예: '3시간 전', '1일 전')
                upload_time_element = video.select_one('div#metadata-line span.inline-metadata-item:last-of-type') 
                upload_time_str = upload_time_element.get_text(strip=True) if upload_time_element else ""

                if not upload_time_str or "전" not in upload_time_str:
                    continue # 업로드 시간이 없거나 '전'이라는 단어가 포함되지 않은 경우 스킵 (라이브 스트림 등)

                # 크롤링 중단 조건 확인 (필터링 조건과 무관하게 '일 전' 동영상 발견 시 중단)
                # 이는 1단계에서 빠르게 너무 오래된 영상 스캔을 멈추는 기준입니다.
                if "일 전" in upload_time_str:
                    days_ago = int(re.search(r'(\d+)', upload_time_str).group(1))
                    # '1일 전'이거나 그 이상이면 중단 (정확한 24시간 필터링은 2단계에서)
                    if days_ago >= 1: 
                        print(f"'{upload_time_str}' 동영상 발견. 1단계 크롤링을 중단합니다.")
                        stop_scrolling = True
                        break 

                # 1단계 업로드 시간 필터링 (대략적인 기준)
                if not is_within_24_hours(upload_time_str):
                    continue

                # 영상 길이 추출 및 필터링
                duration_div = video.select_one('badge-shape.badge-shape-wiz--thumbnail-default div.badge-shape-wiz__text')
                if not duration_div:
                    continue
                duration_str = duration_div.get_text(strip=True)
                video_length_seconds = parse_duration_to_seconds(duration_str)

                if not (MIN_VIDEO_LENGTH_SECONDS <= video_length_seconds <= MAX_VIDEO_LENGTH_SECONDS):
                    continue # 길이 조건 불만족
                
                # 모든 조건 만족 시 저장
                if video_url_suffix and upload_time_str:
                    full_url = f"https://www.youtube.com{video_url_suffix}" # 완전한 URL 생성
                    filtered_video_urls_1st_stage.append({
                        "url": full_url,
                        "upload_time_summary": upload_time_str, # 1단계 필터링용 요약 시간
                    })
                    seen_video_ids.add(video_id) # 최종적으로 추가된 영상만 seen_video_ids에 넣기
                    current_scroll_videos_count += 1

            except (NoSuchElementException, StaleElementReferenceException, AttributeError, IndexError, TypeError) as e:
                continue
            except Exception as e:
                # print(f"처리 중 예상치 못한 오류 발생: {e}") # 디버깅 시에만 사용
                continue

        print(f"현재 스크롤에서 {current_scroll_videos_count}개의 유효한 동영상 후보 발견.")
        if stop_scrolling:
            break

        scroll_count += 1
        time.sleep(1) # 짧은 대기 (과도한 요청 방지)

    print(f"\n1단계 크롤링 완료. 총 {len(filtered_video_urls_1st_stage)}개의 동영상 후보가 수집되었습니다.")
    # 최종적으로 중복 제거
    unique_videos_dict = {video['url']: video for video in filtered_video_urls_1st_stage}
    filtered_video_urls_1st_stage[:] = list(unique_videos_dict.values())
    print(f"최종 1단계 필터링 후 {len(filtered_video_urls_1st_stage)}개의 유니크한 동영상 후보.")

# 1단계 크롤링 실행
if driver:
    crawl_main_page_and_filter_videos(driver)
else:
    print("WebDriver가 초기화되지 않아 1단계 크롤링을 실행할 수 없습니다. 프로그램을 재시작하세요.")

1단계 크롤링 시작: 메인 페이지 동영상 목록 필터링


https://www.youtube.com/channel/UCcQTRi69dsVYHN3exePtZ1A/videos/videos 접속 완료.
페이지 컨텐츠 로딩 완료.

스크롤 1회 시작...
현재 스크롤에서 42개의 유효한 동영상 후보 발견.

스크롤 2회 시작...
현재 스크롤에서 29개의 유효한 동영상 후보 발견.

스크롤 3회 시작...
현재 스크롤에서 28개의 유효한 동영상 후보 발견.

스크롤 4회 시작...
현재 스크롤에서 29개의 유효한 동영상 후보 발견.

스크롤 5회 시작...
현재 스크롤에서 22개의 유효한 동영상 후보 발견.

스크롤 6회 시작...
'1일 전' 동영상 발견. 1단계 크롤링을 중단합니다.
현재 스크롤에서 15개의 유효한 동영상 후보 발견.

1단계 크롤링 완료. 총 165개의 동영상 후보가 수집되었습니다.
최종 1단계 필터링 후 165개의 유니크한 동영상 후보.


In [30]:
# Cell 5: 2단계 - yt-dlp를 이용한 개별 영상 상세 정보 크롤링 함수

def get_youtube_video_details_yt_dlp(video_url):
    """
    yt-dlp를 사용하여 개별 유튜브 영상의 상세 정보를 크롤링합니다.
    """
    ydl_opts = {
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'forcethumbnail': True, # 썸네일 정보를 강제로 가져오도록 시도
        'skip_download': True, # 실제 영상 다운로드 건너뛰기
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            video_info = ydl.extract_info(video_url, download=False)

            title = video_info.get('title', '제목 없음')
            description = video_info.get('description', '설명 없음')
            webpage_url = video_info.get('webpage_url', video_url) # 실제 영상 링크
            
            # 썸네일 링크: 여러 썸네일 중 가장 고화질을 선택하거나, 기본 썸네일을 사용
            thumbnail = video_info.get('thumbnail', None)
            if 'thumbnails' in video_info and video_info['thumbnails']:
                # 일반적으로 마지막 썸네일이 가장 고화질
                thumbnail = video_info['thumbnails'][-1].get('url', thumbnail)

            # KST 형식 날짜/시간 포맷팅
            upload_date_kst_formatted = None
            if 'timestamp' in video_info and video_info['timestamp'] is not None:
                dt_object_utc = datetime.fromtimestamp(video_info['timestamp'], tz=pytz.utc)
                kst_timezone = pytz.timezone('Asia/Seoul')
                dt_object_kst = dt_object_utc.astimezone(kst_timezone)
                upload_date_kst_formatted = dt_object_kst.strftime('%Y-%m-%d %H:%M:%S')
            
            return {
                "title": title,
                "upload_date_kst": upload_date_kst_formatted, # KST 업로드 시간
                "description": description,
                "video_link": webpage_url,
                "thumbnail_link": thumbnail
            }

    except yt_dlp.utils.DownloadError as e:
        print(f"yt-dlp 오류 발생 (URL: {video_url}): {e}")
        return None
    except Exception as e:
        print(f"yt-dlp 처리 중 예상치 못한 오류 발생 (URL: {video_url}): {e}")
        return None

print("2단계 크롤링 함수 정의 완료.")

2단계 크롤링 함수 정의 완료.


In [31]:
# Cell 6: 최종 데이터 크롤링 및 저장 (정확한 24시간 필터링 및 ID 부여)

print("\n최종 데이터 크롤링 시작...")
total_start_time = time.time()

if filtered_video_urls_1st_stage:
    # 현재 시간 (KST) 설정 (yt-dlp 결과와 비교하기 위함)
    kst_timezone = pytz.timezone('Asia/Seoul')
    now_kst = datetime.now(kst_timezone)
    
    # 2단계 크롤링 시 필요한 데이터를 담을 임시 리스트
    # 정렬 후 ID를 부여하기 위해 일단 모든 유효 데이터를 여기에 모음
    temp_final_data_before_id = [] 

    for i, video_data_1st_stage in enumerate(filtered_video_urls_1st_stage):
        video_url = video_data_1st_stage['url']
        print(f"[{i+1}/{len(filtered_video_urls_1st_stage)}] '{video_data_1st_stage.get('upload_time_summary', 'N/A')}' 동영상 상세 정보 크롤링 중: {video_url}")
        
        # Cell 5에서 정의된 yt-dlp 상세 정보 크롤링 함수 호출
        detailed_info = get_youtube_video_details_yt_dlp(video_url)
        
        if detailed_info:
            upload_date_str_kst = detailed_info.get('upload_date_kst', None)
            
            # KST 업로드 시간이 존재하면 정확하게 24시간 필터링
            if upload_date_str_kst:
                try:
                    uploaded_dt_kst = kst_timezone.localize(datetime.strptime(upload_date_str_kst, '%Y-%m-%d %H:%M:%S'))
                    
                    time_difference = now_kst - uploaded_dt_kst
                    
                    # 24시간(86400초) 이상이면 최종 데이터에 추가하지 않음
                    if time_difference.total_seconds() >= 24 * 3600:
                        print(f" -> '{upload_date_str_kst}' (KST) 영상은 24시간을 초과하여 제외합니다.")
                        time.sleep(0.5) # 요청 간 짧은 대기
                        continue # 이 영상은 최종 목록에서 제외하고 다음 영상으로 넘어감
                    
                except ValueError:
                    print(f" -> 경고: KST 업로드 시간 '{upload_date_str_kst}' 파싱 오류. 필터링하지 않고 포함합니다.")
            else:
                print(f" -> 경고: YT-DLP에서 KST 업로드 시간을 가져오지 못했습니다. 이 영상은 필터링하지 않고 포함합니다.")

            # 24시간 이내인 경우에만 임시 리스트에 추가
            temp_final_data_before_id.append(detailed_info)
            print(f" -> 제목: {detailed_info.get('title', 'N/A')}, 업로드: {detailed_info.get('upload_date_kst', 'N/A')}")
        else:
            print(f" -> 상세 정보 크롤링 실패 또는 정보 부족: {video_url}")
        
        time.sleep(0.5) # 개별 영상 요청 간 짧은 지연

    # 최종 필터링된 데이터를 최신순으로 정렬하고 순차적인 ID 부여
    if temp_final_data_before_id:
        # 'upload_date_kst' 문자열을 datetime 객체로 변환하여 정확하게 최신순으로 정렬
        final_video_data.sort(
            key=lambda x: datetime.strptime(x['upload_date_kst'], '%Y-%m-%d %H:%M:%S') if x.get('upload_date_kst') else datetime.min,
            reverse=True # 내림차순 (최신 영상이 먼저 오도록)
        )
        
        # 정렬된 순서대로 순차적인 ID 부여
        for i, item in enumerate(temp_final_data_before_id):
            item['id'] = i + 1 # 1부터 시작하는 순번 부여
            final_video_data.append(item) # 최종 리스트에 추가

    print(f"\n총 {len(final_video_data)}개의 유효한 영상 상세 정보 크롤링 완료.")

    # JSON 파일로 저장
    output_filename = "크롤링_샘플_데이터.json"
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(final_video_data, f, ensure_ascii=False, indent=4)
        print(f"크롤링된 데이터가 '{output_filename}' 파일에 성공적으로 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}")
else:
    print("1단계 필터링된 영상이 없어 2단계 크롤링을 진행하지 않습니다.")

total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"\n**전체 크롤링 작업 소요 시간**: {total_elapsed_time:.4f} 초")

# WebDriver 종료
if driver:
    driver.quit()
    print("WebDriver 종료.")


최종 데이터 크롤링 시작...
[1/165] '5분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=Y_MwYUrX1c4
 -> 제목: ‘콜롬비아 의원 저격범’은 14살…“무기, 미국서 밀반입” [맵 브리핑] / KBS  2025.06.10., 업로드: 2025-06-10 15:30:17
[2/165] '7분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=ehxhO2REbPo
 -> 제목: LA 시위서 멕시코인 ‘줄체포’…급기야 나선 멕시코 대통령 [맵 브리핑] / KBS  2025.06.10., 업로드: 2025-06-10 15:28:55
[3/165] '9분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=kAwnqs-S8tM
 -> 제목: [자막뉴스] "나토 가입포기 없다면 핵전쟁"…러시아 '여름 대공세' 시작했나 / KBS 2025.06.10., 업로드: 2025-06-10 15:26:30
[4/165] '17분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=xmqz2r54Z4A
 -> 제목: 민주당 원내대표 후보자 합동 토론회…국민의힘, 원외 당협위원장 간담회 개최 / KBS  2025.06.10., 업로드: 2025-06-10 15:19:04
[5/165] '17분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=gN8rah3CboY
 -> 제목: 조계사 국제회의장 화재 1시간 반만에 완진…인명·문화유산 피해 없어 / KBS  2025.06.10., 업로드: 2025-06-10 15:19:02
[6/165] '17분 전' 동영상 상세 정보 크롤링 중: https://www.youtube.com/watch?v=kEvWPrbE3EU
 -> 제목: 이 대통령 주재 국무회의서 ‘3대 특검’ 의결…한중정상 통화 / KBS  2025.